In [22]:
import comtypes.client

import pandas as pd

import numpy as np
from math import sqrt

from itertools import accumulate

In [ ]:
class EstructuraParametrica(object):
    name_project = "Estructura Paramétrica"
    def __init__(self, dx, dy, hz, tn, path):
        self.dx = dx
        self.dy = dy
        self.hz = hz
        self.tn = tn            # espesor de la losa
        # self.CM_load = CM_load
        # self.CV_load = CV_load
        self.path = path        # ruta de datos requeridos

        self.x = [0] + list(accumulate(self.dx))
        self.y = [0] + list(accumulate(self.dy))
        self.z = [0] + list(accumulate(self.hz))
        print("Coordenadas X:", self.x, "\nCoordenadas Y:", self.y, "\nCoordenadas Z:", self.z)

        self.smodel = self.Connect_ETABS()

        # self.connect_to = 2
        # state, connect_software = self.connect_default()
        # connect_software.ApplicationStart()  # Inicia la aplicación ETABS o SAP2000
        # self.smodel = connect_software.SapModel
        
        # response = self.smodel.InitializeNewModel()
        # response = self.smodel.File.NewBlank()
                
        # self.set_units() # self.set_units
        self.Sections(self.path)
        self.Load_Combinations()
        self.Frames(self.x, self.y, self.z, self.path)
        self.Slabs(self.tn, self.x, self.y, self.z)
        # self.Loads(self.CM_load, self.CV_load)

        self.smodel.View.RefreshView()

    def Connect_ETABS(self):
        helper = comtypes.client.CreateObject('ETABSv1.Helper')             # Maneja las conexiones con ETABS
        helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)        # Accede a métodos avanzados para interactuar con ETABS
        ETABSObject = helper.GetObject("CSI.ETABS.API.ETABSObject")         # Se conecta a ETABS 
        smodel = ETABSObject.SapModel                                       # Accede al modelo estructural actualmente cargado en ETABS
        return smodel

    def connect_default(self):
        connect_to_app = None
        helper = comtypes.client.CreateObject('ETABSv1.Helper')
        if self.connect_to == 1: helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)
        if self.connect_to == 2: helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper)
        try:
            connect_to_app = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject")
        except (OSError, comtypes.COMError):
            # print(f"Cannot start a new instance of the program({self.app_csi}).")
            return (False, connect_to_app)
        # print(f"Coneccion establecida para {self.app_csi}!.")
        return (True, connect_to_app)
    
    def set_units(self, units):
        self.smodel.SetPresentUnits(units)
    
    def Frames(self, x, y, z, path):
        beams_x = pd.read_excel(path, sheet_name='VIGAS_X')
        beams_y = pd.read_excel(path, sheet_name='VIGAS_Y')
        columns = pd.read_excel(path, sheet_name='COLUMNAS')        

        for i in range(len(x)):
            for j in range(len(y)):
                for k in range(len(z)-1):
                    # Columnas
                    node = f'NODO {str(int(i+1))}'
                    ret = self.smodel.FrameObj.AddByCoord(
                        x[i], y[j], z[k], 
                        x[i], y[j], z[k+1], 
                        "", columns[node][j]
                    )                    
        for k in range(1, len(z)):
            for j in range(len(y)):
                for i in range(len(x)-1):
                    # Vigas en X
                    span = 'TRAMO '+ str(int(j+1))
                    ret = self.smodel.FrameObj.AddByCoord(x[i], y[j], z[k], x[i+1], y[j], z[k], "", beams_x[span][i])

            for i in range(len(x)):
                for j in range(len(y)-1):
                    # Vigas en Y
                    span = 'TRAMO '+ str(int(i+1))
                    ret = self.smodel.FrameObj.AddByCoord(x[i], y[j], z[k], x[i], y[j+1], z[k], "", beams_y[span][j])
        self.smodel.View.RefreshView()
        num_frames = self.smodel.FrameObj.Count()
        print("Total de elementos de marco en el modelo:", num_frames)

    def Slabs(self, tn, x, y, z):
        Lname = "L"+ str(tn)
        ret = self.smodel.PropArea.SetSlab(Lname, 0, 1, "fc = 280 kg/cm2", tn)

        for k in range(1, len(z)):
            for i in range(len(x)-1):
                for j in range(len(y)-1):
                    xc = [x[i], x[i + 1], x[i + 1], x[i]]
                    yc = [y[j], y[j], y[j + 1], y[j + 1]]
                    zc = [z[k]]*4
                    ret = self.smodel.AreaObj.AddByCoord(4, xc, yc, zc, "", Lname)

    def Concrete_material(self, fc):
        Conc_name = "fc = " + str(fc) + " kg/cm2"
        ret = self.smodel.PropMaterial.SetMaterial(Conc_name, 2)
        ret = self.smodel.PropMaterial.SetOConcrete_1(
            Conc_name, 
            fc, 
            False, 
            0, 
            1, 
            2, 
            0.0022, 
            0.0052, 
            -0.1, 
            0, 
            0
            )
        ret = self.smodel.PropMaterial.SetMPIsotropic(
            Conc_name, 
            15100*sqrt(fc), 
            0.2, 
            0.0000099
            )
        return Conc_name
    
    def Rebar_material(self, fy):
        Rebar_name = "fy = " + str(fy) + " kg/cm2"
        ret = self.smodel.PropMaterial.SetMaterial(Rebar_name, 6)
        ret = self.smodel.PropMaterial.SetORebar_1(
            Rebar_name, 
            fy, 
            6300, 
            fy, 
            6300, 
            2, 
            2, 
            0.02, 
            0.1, 
            -0.1, 
            False
            )
        ret = self.smodel.PropMaterial.SetMPIsotropic(
            Rebar_name, 
            2*(10**6), 
            0.2, 
            0.0000117
            )
        return Rebar_name
    
    def Beams_section(self, Name, b, h, fc, fy):
        Cname = self.Concrete_material(fc)
        Rname = self.Rebar_material(fy)
        ret = self.smodel.PropFrame.SetRectangle(Name, Cname, h, b)
        ret = self.smodel.PropFrame.SetRebarBeam(Name, Rname, Rname, 0, 0, 0, 0, 0, 0)
    
    def Columns_section(self, Name, b, h, fc, fy):
        Cname = self.Concrete_material(fc)
        Rname = self.Rebar_material(fy)
        ret = self.smodel.PropFrame.SetRectangle(Name, Cname, h, b)
        ret = self.smodel.PropFrame.SetRebarColumn(Name, Rname, Rname, 2, 2, 2, 10, 0, 0, "#10", "#5", 4, 0, 0, False)
    
    def Sections(self, path):
        data = pd.read_excel(path, sheet_name='DATOS')
        for _, row in data.iterrows():
            if row['Tipo'] == 'Viga':       self.Beams_section(row['Nombre'], row['b'], row['h'], row['fc'], row['fy'])
            elif row['Tipo'] == 'Columna':  self.Columns_section(row['Nombre'], row['b'], row['h'], row['fc'], row['fy'])
    
    def Load_Combinations(self):       
        # Definición del tipo de carga y el multiplicador
        ret = self.smodel.LoadPatterns.Add('P.PROPIO', 1, 1)      # Peso Propio
        ret = self.smodel.LoadPatterns.Add('CM', 1, 0)            # Carga muerta
        ret = self.smodel.LoadPatterns.Add('CV', 3, 0)            # Carga viva
    
    def Loads(self, CM_load, CV_load):
        ret, area_names, _ = self.smodel.AreaObj.GetNameList()
        for name in area_names:
            ret = self.smodel.AreaObj.SetLoadUniform(name, "CM", -CM_load, 2, True, "Local", 0)
            ret = self.smodel.AreaObj.SetLoadUniform(name, "CV", -CV_load, 2, True, "Local", 0)

    def close_software(self):
        self.smodel.File.Save(self.name_project + ".edb")   # Guarda el modelo

        # self.smodel.ApplicationExit(False)                # Cierra ETABS sin guardar cambios adicionales
        print("No se pudo cerrar") if (self.smodel.ApplicationExit(False)) else print("Se cerro con exito.")
        # self.connect_to_app.Quit()  # Finaliza la conexión con ETABS
        self.helper = None  # Libera el objeto helper

        smodel= None
        del smodel
        
        return 0

<center>
<img src="images/ModeloA.drawio.svg" alt='' width="80%"/>
</center>

In [44]:
# Parametros
# ----------

# modelA = {
#     "geometria": {
#         'distancia de vanos en X': [500, 400, 500],
#         'distancia de vanos en Y': [400, 300, 300, 400],
#         'alturas de entrepiso': [280] + numStorys*[300],
#     },
#     "Cargas": {
#         'Carga muerta': 100e-4,
#         'Carga viva': 200e-4
#     },
#     "Unidades": {}
# }

numStorys = 1                       # Numero de pisos --- si es 0 tendra un piso y ...
dx = [500, 400, 500]                # Distancia de vanos - Eje X
dy = [400, 300, 300, 400]           # Distancia de vanos - Eje Y
hz = [280] + numStorys*[300]        # Atluras de entrepisos

units = 14                          #  kgf_cm_C = 14, kN_m_C = 6
path = 'model02/Modelo_Parametrico.xlsx'   # Nombre del Archivo

tn = 20     # Espesor de losa

CM_load = 100e-4                    # Carga Muerta
CV_load = 200e-4                    # Carga Viva

estructura01 = EstructuraParametrica(dx, dy, hz, tn, path)
estructura01.set_units(units)
estructura01.Loads(CM_load, CV_load)


Coordenadas X: [0, 500, 900, 1400] 
Coordenadas Y: [0, 400, 700, 1000, 1400] 
Coordenadas Z: [0, 280, 580]


TypeError: call takes exactly 5 arguments (12 given)

In [ ]:
# estructura01.close_software()
# exit(-1)

## Referencia.

- https://youtu.be/hYBF3mG95hk